# Assemble and save data matrices

In [1]:
import numpy as np
import torch as pt
import matplotlib.pyplot as plt
from stl import mesh
from flowtorch.data import FOAMDataloader, mask_box

plt.rcParams['figure.dpi'] = 160

run = "/media/andre/Elements/naca0012_shock_buffet/run/"
output = "./output/"
!mkdir -p output

## 2D simulation

In [3]:
chord = 0.6010500

path = run + "rhoCF_set1_alpha4_saiddes_ref2/"
loader = FOAMDataloader(path)
write_times = loader.write_times[1:]
fields = loader.field_names[write_times[0]]
vertices = loader.vertices
weights = loader.weights
print("Available snapshots: ", len(write_times))
print("Available fields: ", fields)

Available snapshots:  2500
Available fields:  ['alphat', 'Ma', 'nut', 'nuTilda', 'p', 'rho', 'T', 'total(p)_coeff', 'U', 'yPlus']


In [4]:
vertices /= chord
mask = mask_box(vertices[:, :2], [-1, -1], [3, 1])
n_points = mask.sum().item()

x = pt.masked_select(vertices[:, 0], mask)
y = pt.masked_select(vertices[:, 1], mask)
weights = pt.masked_select(weights, mask)
print("selected points: ", n_points)

selected points:  168508


In [5]:
pt.save(x, f'{output}x_masked_2D_ref2.pt')
pt.save(y, f'{output}y_masked_2D_ref2.pt')
pt.save(weights, f'{output}vol_masked_2D_ref2.pt')

In [6]:
times_num = pt.tensor([float(t) for t in write_times])
pt.save(times_num, f'{output}t_2D_ref2.pt')

In [9]:
write_times[309], write_times[2309]

('0.031', '0.231')

In [10]:
time_window = write_times[309:2310]
Ux = pt.zeros((n_points, len(time_window)))
Uy = pt.zeros_like(Ux)
Ma_loc = pt.zeros_like(Ux)
for i, time in enumerate(time_window):
    print("\r", f"Processing snapshot {i+1}/{len(time_window)}", end="")
    U_i, Ma_i = loader.load_snapshot(["U", "Ma"], time)
    Ux[:, i] = pt.masked_select(U_i[:, 0], mask)
    Uy[:, i] = pt.masked_select(U_i[:, 1], mask)
    Ma_loc[:, i] = pt.masked_select(Ma_i, mask)

 Processing snapshot 2001/2001

In [11]:
alpha = 1.0
kappa = 1.4
sqWeights = weights.sqrt().unsqueeze(-1)
data_matrix = pt.zeros((3*n_points, len(time_window)))
data_matrix[:n_points, :] = Ux * sqWeights
data_matrix[n_points:2*n_points, :] = Uy * sqWeights
data_matrix[2*n_points:, :] = 2.0*alpha/(kappa-1.0) * pt.sqrt(Ux**2 + Uy**2) / Ma_loc * sqWeights

In [12]:
pt.save(data_matrix, f'{output}comp_data_matrix_2D_ref2.pt')

## 3D simulation

In [13]:
path = run + "rhoCF_set1_alpha4_saiddes_ref0_z25/"
loader = FOAMDataloader(path)
write_times = loader.write_times[1:]
fields = loader.field_names[write_times[0]]
loader._case._distributed = False
vertices = loader.vertices
volumes = loader.weights
print("Available snapshots: ", len(write_times))
print("Available fields: ", fields)

Loading precomputed cell centers and volumes from constant/
Available snapshots:  1255
Available fields:  ['alphat', 'Ma', 'nut', 'nuTilda', 'p', 'rho', 'T', 'total(p)_coeff', 'U', 'yPlus']


In [14]:
vertices /= chord
# pick a slice from the middle spanwise position
mask = mask_box(vertices, [-1, -1, 0.0024042/chord], [3, 1, 0.0072126/chord])
n_points = mask.sum().item()

x = pt.masked_select(vertices[:, 0], mask)
y = pt.masked_select(vertices[:, 1], mask)
weights = pt.masked_select(volumes, mask)
print("selected points: ", n_points)

selected points:  79706


In [15]:
pt.save(x, f'{output}x_masked_3D_ref0.pt')
pt.save(y, f'{output}y_masked_3D_ref0.pt')
pt.save(weights, f'{output}vol_masked_3D_ref0.pt')

In [16]:
times_num = pt.tensor([float(t) for t in write_times])
pt.save(times_num, f'{output}t_3D_ref0.pt')

In [18]:
write_times[322], write_times[1011]

('0.0326', '0.1015')

In [19]:
time_window = write_times[322:1012]
Ux = pt.zeros((n_points, len(time_window)))
Uy = pt.zeros_like(Ux)
Uz = pt.zeros_like(Ux)
Ma_loc = pt.zeros_like(Ux)
for i, time in enumerate(time_window):
    print("\r", f"Processing snapshot {i+1}/{len(time_window)}", end="")
    U_i, Ma_i = loader.load_snapshot(["U", "Ma"], time)
    Ux[:, i] = pt.masked_select(U_i[:, 0], mask)
    Uy[:, i] = pt.masked_select(U_i[:, 1], mask)
    Uz[:, i] = pt.masked_select(U_i[:, 2], mask)
    Ma_loc[:, i] = pt.masked_select(Ma_i, mask)

 Processing snapshot 690/690

In [20]:
sqWeights = weights.sqrt().unsqueeze(-1)
data_matrix = pt.zeros((4*n_points, len(time_window)))
data_matrix[:n_points, :] = Ux * sqWeights
data_matrix[n_points:2*n_points, :] = Uy * sqWeights
data_matrix[2*n_points:3*n_points, :] = Uz * sqWeights
data_matrix[3*n_points:, :] = 2.0*alpha/(kappa-1.0) * pt.sqrt(Ux**2 + Uy**2 + Uz**2) / Ma_loc * sqWeights

In [ ]:
pt.save(data_matrix, f'{output}comp_data_matrix_3D_ref0.pt')